# TP2: Architectural Analysis

Extract topological data from MTG. 

These data will be used later for simulation of AppleTree architecture and development.

For that we will follow different steps:
* Load one of the tree digitised MTG
* Explore the MTG as an architectural database 
  - number of vertices
  - class of vertices
  - number of scales
  - properties
  - ...
  
You can find the **documentation** of the MTG on the web : 

[MTG documentation](http://mtg.readthedocs.io/en/latest/user/api_mtg.html#overview)

## Load an MTG file

In [ ]:
from path import Path
# load MTG files
data = Path('data')

breaburn, fuji1, fuji2= data.glob('apple*.mtg')
breaburn

In [ ]:
s =!cat $breaburn
print '\n'.join(s[:90])

## Load MTGs in memory

- Vertex identifier: vid
- MTG data structure : g


In [ ]:
%matplotlib notebook
import pandas as pd

from openalea.mtg import *
gb = MTG(breaburn)
g1 = MTG(fuji1) 
g2 = MTG(fuji2) 

g = g2
#print g
#g.display(max_scale=3)


In [ ]:
# Print the number of scales
nb_scales = g.nb_scales()

# Print the vertices at different scales
for scale in range(1, nb_scales):
    print 'Nb vertices at scale ', scale, ': ', g.nb_vertices(scale=scale)
    print 'Classes : ', list(set(g.class_name(vid) for vid in g.vertices(scale=scale)))


# Classes represent modularities
## Scale 1: P = Plant

## scale 2: Axes 
- S: Short 
- R : Long  
- M : Medium 

## scale 3 : Growth Unit
- I : Inflorescence
- U : Long GU
- D : Short GU
- W : Medium GU

## Scale 4: phytomer
- E: vegetative
- F: fruit

Fruit are not a modularity but we can encore specific attributes

In [ ]:
# Select all the vertices of each classes

classes = list(set(g.class_name(vid) for vid in g.vertices() if g.class_name(vid)))
print classes

def vertices(g, class_name='P'):
    return [vid for vid in g.vertices() if g.class_name(vid)==class_name]

vids_E = vertices(g, 'E')
print 'Nb E', len(vids_E)


# Exercise : Get properties on the MTG and on each vertices

Get the following values using methods on g (MTG):
* `property_names`: all the property names
* `property`: get one property
* access to one property (eg `hauteur`)
* access to all the properties of one vertex (use g[vid] with vid = vids_E[0]
* use the method `nid = g.node(vid)` for the same vid and access to some properties (nid.hauteur)
* print the `label` and the `index` as well as the `class_name` of one vertex (using `g.index` or `node` API)

In [ ]:
# Get one vertex
vid = vids_E[0]

# Properties on the MTG: this exclude all the topological properties
#print g.property_names()

# Retrieve one property for the MTG (dict)
# TODO

# There are several methods to get all the properties of a given vertex (eg vid)
# Use g[vid], g.node(vid), access to one property using g.node(vid).hauteur

# get the label, index and class_name


## Exercise : Compute at GU scale : for each class and for each year, 

the number of growth unit by type and year

- the class of a vertex : g.class_name(v)
- the year is the encoded as the index: $label = class + index$
year = g.index(v) or node.index()

**NB**: In Python a *set* is a collection that return a unique value of each element

In [ ]:

# Extract the years at scale 3
# The year is coded in the index
years = set(int(g.index(vid)) for vid in g.vertices(scale=3))
print sorted(years)

#print [g.class_name(vid) for vid in g.vertices(scale=3)]

classes = set(g.class_name(vid) for vid in g.vertices(scale=3))
print classes

d = {}

for _class in classes:
    d[_class] = {}
    for _year in years:
        d[_class][_year] = 0
        
################################
# TODO
# Modify the following code

for vid in g.vertices(scale=3):
    _class = 'I' # replace this line by using class_name of vid
    _year = 94 # TODO : use g.index to have the year
    d[_class][_year] += 1

################################


print d

# Build a Data frame for the dict
df = pd.DataFrame.from_dict(d)
df.plot()
df

In [ ]:
# Export to a pandas dataframe for processing in R

# We need a dict with columns class, year and number

dd = datadict = {'class':[], 'year': [], 'number':[]}
for k in d:
    for y in d[k]:
        dd['class'].append(k)
        dd['year'].append(y)
        dd['number'].append(d[k][y])
        

df = pd.DataFrame(data=dd, index=None, columns=['class', 'year', 'number'])
df.sort_values(['class', 'year'])
df.to_csv('result_GU_num.csv', index=None)
# You can view the content by uncomment the following line
#df

## Exercise : Extract for each Growth Unit the number of metamer (components at scale 4)

**TODO**: 
* Extract all the vertices at GU scale
* For each GU, extract the number of components (use `nb_components`) 
* Implement the function `data`

In [ ]:
GUs = g.vertices(scale=3)

################################
# TODO

def data(vid):
    """ TODO 
    Return vid, the class name and the number of metamers per GU
    """
    class_name = 'I' # TODO
    nb_metamers = 1 # nb metamers is the number of components of a GU
    return [vid, class_name, nb_metamers]

################################

nb_metamers = [data(vid) for vid in GUs]

df_gu = pd.DataFrame(nb_metamers, columns=['vid', 'class', 'number'])

df_gu.to_csv('result_metamer_num.csv', index=None)
#df_gu

## Exercise : Extract the number of ramification from each growth unit

**Help**: To extract the number of ramification, use  `g.Sons(vid, EdgeType='+')`

In [ ]:
# You can use the method Sons on MTG

################################
# TODO: Implement the following line
def data_ramif(vid):
    """ TODO 
    Return vid, the class name and the number of metamers per GU
    """
    class_name = g.class_name(vid)
    nb_ramif = 1 # replace this line by the number of ramifs for vid using g.Sons
    return vid, class_name, nb_ramif

nb_ramifs = [data_ramif(vid) for vid in GUs]



################################

df_ramif = pd.DataFrame(nb_ramifs, columns=['vid', 'class', 'nb_ramifs'])
#df_ramif.to_csv('result_branch_num.csv', index=None)
#df_ramif

## Exercise: For each vertex at scale Metamer (scale =4), population of ramifications and position

- vertex of the complex
- Class of complex
- year of complex
- position / rank of each metamer
- if no ramification, put B (blind) else the class of the complex of its child


In [ ]:
def rank(v):
    """ Returns the rank of a vertex within its complex. """
    cid = g.complex(v)
    root_in_complex = g.component_roots(cid)[0]
    return g.Height(root_in_complex, v)

ramif_location = []

################################
# TODO

for vid in g.vertices(scale=4):
    # TODO : implement the following lines
    complex_id = 1
    c_class = 'I' # class of the complex
    c_year = 94 # year of the complex
    metamer_rank = 1 #
    ramifs = [] # list of ramifications of the vid
    nb_ramifs = 0 # number of ramification
    
    # Class of the complex of the ramif:
    ramif_class = g.class_name(g.complex(ramifs[0])) if (nb_ramifs != 0) else 'B' 
    
    ramif_location.append((complex_id, c_class, c_year, metamer_rank, nb_ramifs, ramif_class))

################################

    
df_pop = pd.DataFrame(ramif_location, columns=['cid', 'class', 'year', 'metamer_rank', 'number_ramif', 'ramif_class'])
#df_pop.to_csv('result_ramif_location.csv', index=None)
    
#df_pop

## Exercise : For each vertex at scale Metamer, population of ramifications and position

- vertex of the complex
- Class of complex
- year of complex
- position / rank of each metamer
- if no ramification, put B (blind) else the class of the complex of its child


In [ ]:
ramif_location = []

################################
# TODO 

for vid in g.vertices(scale=4):
    complex_id = g.complex(vid)
    c_class = g.class_name(complex_id)
    c_year = int(g.index(complex_id))
    metamer_rank = rank(vid)
    ramifs = g.Sons(vid, EdgeType='+')
    if ramifs and g.class_name(g.complex(ramifs[0])) == "I":
        # Todo: Retrieve the vegetative GU following the inflorescence (due to sympodial growth: EdgeType='+')
        # Get the type of the first vegetative GU
        # And the number of ramifications
        nb_ramifs = 0
        ramif_class = 'I'
    else:
        nb_ramifs = 0   
        ramif_class = g.class_name(g.complex(ramifs[0])) if (nb_ramifs != 0) else 'B' 
    
    ramif_location.append((complex_id, c_class, c_year, metamer_rank, nb_ramifs, ramif_class))

################################

#df_pop = pd.DataFrame(ramif_location, columns=['cid', 'class', 'year', 'metamer_rank', 'number_ramif', 'ramif_class'])
#df_pop.to_csv('result_ramif_location_variant.csv', index=None)

## Exercise : For each year, and each type  give the number of successor of each type

For all the Growth Unit of a given class and  year : 
- get the number of successor (its child whose edge_type is '<') of each type

There is a particular case for inflorescences:
- In case of inflo GrowthUnit (class I), the successor is a ramification (edge-type == '+') due to its sympodial nature.



In [ ]:
# {'D', 'I', 'U', 'W'} with I : inflorescence
classes = set(g.class_name(vid) for vid in g.vertices(scale=3))


d = {}

for c in classes:
    d[c] = {}
    for y in years:
        d[c][y] = {}
        for sc in classes:
            d[c][y][sc] = 0

        
def my_successors(vid):
    if g.class_name(vid) != 'I':
        cid = g.Successor(vid)
        if cid is not None:
            return [cid]
        else:
            return []
    else:
        return g.children(vid)

################################
# TODO

for vid in GUs:
    _class = g.class_name(vid)
    _year = 94
    kids = my_successors(vid)
    for cid in kids:
        s_class = 'I'
        d[_class][_year][s_class] += 1

################################

# This part is used transform the dict into a list of rows
# 
successors = []
for klass in d:
    for _year in d[klass]:
        data = [klass, _year]
        for succ_klass in classes:
            data.append(d[klass][_year][succ_klass])
        successors.append(data)

columns= ['class', 'year']+list(classes)

df_successors = pd.DataFrame(successors, columns=columns)
df_successors.to_csv('result_successors.csv', index=None)
#df_successors

## Plot the geometry of the Tree

Open an other notebook by clicking below:

[Plot the trees with PlantFrame Algorithm](3D_plot.ipynb)

![3dtree](breaburn_3d.png "3dtree")

# Solutions are [here](MTG_Analysis_solution.ipynb)